# ボストンのMEDV（物件価格の中央値）の回帰

## データセットの準備

In [1]:
from sklearn.datasets import load_boston

dataset = load_boston()

In [2]:
boston_data = dataset.data
boston_target = dataset.target

In [3]:
boston_data.shape

(506, 13)

In [4]:
boston_target.shape

(506,)

### データセットの分割

In [32]:
# データセットを分割する関数の読み込み
from sklearn.model_selection import train_test_split

# 訓練用データセットとテスト用データセットへの分割
boston_data_train, boston_data_test, boston_target_train, boston_target_test = train_test_split(boston_data, boston_target, test_size=0.3, random_state=0)

# 分割とともにdataframe型からndarray型に変換している
type(boston_data_train)

numpy.ndarray

## モデル・目的関数・最適化手法の定義

In [33]:
from sklearn.linear_model import LinearRegression

# 最小二乗法を行うclassのインスタンス化
# 内部で目的関数や最適化手法も行なっている
reg_model = LinearRegression()

## モデルの訓練

In [34]:
# モデルの重みwとバイアスbが求まる
reg_model.fit(boston_data_train, boston_target_train)

LinearRegression()

In [35]:
# 訓練後のパラメータ w
reg_model.coef_

array([-1.21310401e-01,  4.44664254e-02,  1.13416945e-02,  2.51124642e+00,
       -1.62312529e+01,  3.85906801e+00, -9.98516565e-03, -1.50026956e+00,
        2.42143466e-01, -1.10716124e-02, -1.01775264e+00,  6.81446545e-03,
       -4.86738066e-01])

In [36]:
# 訓練後のバイアス b
reg_model.intercept_

37.93710774183228

In [37]:
# 決定係数で精度の測定
reg_model.score(boston_data_train, boston_target_train)

0.7645451026942549

## 訓練済みのモデルの推論

In [38]:
# テストデータから一つ取り出し、推論を実行
reg_model.predict(boston_data_test[:1])

array([24.9357079])

In [39]:
# このテストデータの目標値
boston_target_test[0]

22.6

## テスト用データセットを用いた評価

In [40]:
# テストデータを使って決定係数Rを求める
reg_model.score(boston_data_test, boston_target_test)

0.6733825506400195

訓練データの場合のR値は約0.765だったのに対し、テストデータの場合は約0.673になっている。このことから、モデルが訓練データにのみ当てはまりすぎている過学習が起きていると考えられる。

## 前処理（データセットの準備における改善）

### 標準化

In [41]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [42]:
# 訓練用データセットの平均と分散の計算
scaler.fit(boston_data_train)

StandardScaler()

In [43]:
# 平均
scaler.mean_

array([3.35828432e+00, 1.18093220e+01, 1.10787571e+01, 6.49717514e-02,
       5.56098305e-01, 6.30842655e+00, 6.89940678e+01, 3.76245876e+00,
       9.35310734e+00, 4.01782486e+02, 1.84734463e+01, 3.60601186e+02,
       1.24406497e+01])

In [44]:
# 分散
scaler.var_

array([6.95792305e+01, 5.57886665e+02, 4.87753572e+01, 6.07504229e-02,
       1.33257561e-02, 4.91423928e-01, 7.83932705e+02, 4.26314655e+00,
       7.49911344e+01, 2.90195600e+04, 4.93579208e+00, 7.31040807e+03,
       4.99634123e+01])

In [45]:
# 訓練データ、テストデータの標準化
boston_data_train_scaled = scaler.transform(boston_data_train)
boston_data_test_scaled  = scaler.transform(boston_data_test)

In [46]:
reg_model = LinearRegression()

# モデルの訓練
reg_model.fit(boston_data_train_scaled, boston_target_train)

LinearRegression()

In [47]:
# 精度の検証（訓練データ）
reg_model.score(boston_data_train_scaled, boston_target_train)

0.7645451026942549

In [48]:
# 精度の検証（テストデータ）
reg_model.score(boston_data_test_scaled, boston_target_test)

0.6733825506400195

## 正規化

In [49]:
from sklearn.preprocessing import PowerTransformer

scaler = PowerTransformer()
scaler.fit(boston_data_train)

boston_data_train_scaled = scaler.transform(boston_data_train)
boston_data_test_scaled = scaler.transform(boston_data_test)

reg_model = LinearRegression()
reg_model.fit(boston_data_train_scaled, boston_target_train)

/Users/H.Kondo/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:3237: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


LinearRegression()

In [50]:
# 精度の検証（訓練データ）
reg_model.score(boston_data_train_scaled, boston_target_train)

0.7859862562650238

In [51]:
# 精度の検証（テストデータ）
reg_model.score(boston_data_test_scaled, boston_target_test)

0.700285655245619

## パイプライン化

In [52]:
from sklearn.pipeline import Pipeline

# パイプラインの作成 (scaler -> svr)
pipeline = Pipeline([
    ('scaler', PowerTransformer()),
    ('reg', LinearRegression())
])

In [53]:
# scaler および reg を順番に使用
pipeline.fit(boston_data_train, boston_target_train)

/Users/H.Kondo/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:3237: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


Pipeline(steps=[('scaler', PowerTransformer()), ('reg', LinearRegression())])

In [54]:
# 訓練用データセットを用いた決定係数の算出
pipeline.score(boston_data_train, boston_target_train)

0.7859862562650238

In [55]:
# テスト用データセットを用いた決定係数の算出
linear_result = pipeline.score(boston_data_test, boston_target_test)

linear_result

0.700285655245619